In [ ]:
pip install face_recognition

In [ ]:
# import the necessary packages
import pickle
import cv2
import face_recognition
from google.colab import drive
from google.colab.patches import cv2_imshow  #cv2.imshow crashes Jupyter session

In [ ]:
drive.mount("/content/gdrive")

In [ ]:
% cd /content/gdrive/My Drive/FaceRecog_AltMethod/ForGit

In [ ]:
# load the known faces and embeddings
print("[INFO] loading encodings...")
data_known_encoding = pickle.loads(open("encodings.pickle", "rb").read())

# load the input image and convert it from BGR to RGB
#image_unknown = cv2.imread("testimages/test1.jpg")
image_unknown = cv2.imread("testimages/test2.JPG")
rgb_unknown = cv2.cvtColor(image_unknown, cv2.COLOR_BGR2RGB)

# detect the (x, y)-coordinates of the bounding boxes corresponding
# to each face in the input image, then compute the facial embeddings
# for each face
print("[INFO] recognizing faces...")
#get the face encodings for the UNKNOWN input image
boxes_unknown = face_recognition.face_locations(rgb_unknown,model="cnn")
encodings_unknown = face_recognition.face_encodings(rgb_unknown, boxes_unknown)

# initialize the list of names for each face detected
names = []

# loop over the facial embeddings
for encoding_unknown in encodings_unknown:
	matches = face_recognition.compare_faces(data_known_encoding["encodings"],	encoding_unknown)
	name = "Unknown"

  	# check to see if we have found a match
	if True in matches:		
		matchedIdxs = [i for (i, b) in enumerate(matches) if b]
		counts = {}
		# loop over the matched indexes and maintain a count for
		# each recognized face face
		for i in matchedIdxs:
			name = data_known_encoding["names"][i]
			counts[name] = counts.get(name, 0) + 1
		name = max(counts, key=counts.get)
	
	# update the list of names
	names.append(name)
 
#loop over the bounding boxes and labeled names for each person and draw them on our output image for visualization purposes
# loop over the recognized faces
for ((top, right, bottom, left), name) in zip(boxes_unknown, names):
	# draw the predicted face name on the image
	cv2.rectangle(image_unknown, (left, top), (right, bottom), (0, 255, 0), 2)
	y = top - 15 if top - 15 > 15 else top + 15
	cv2.putText(image_unknown, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,	0.75, (0, 255, 0), 2)

# show the output image
cv2_imshow(image_unknown)
cv2.waitKey(0)